In [9]:
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
from tqdm import tqdm
import random

In [2]:
n_particles = 1
system = mm.System()
mass   = 14.007 * unit.amu # masa del átomo N
system.addParticle(mass)

force = mm.CustomExternalForce('Eo*(C*x^4-x^2)+M*x + (K/2.0)*(y^2+z^2)')
force.addGlobalParameter('K', 35 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('Eo', 6.0 * unit.kilocalories_per_mole/unit.angstrom**2)
force.addGlobalParameter('C', 0.4 * 1/unit.angstrom**2)
force.addGlobalParameter('M', 0.25 * unit.kilocalories_per_mole/unit.angstrom)
force.addParticle(0, [])
system.addForce(force)

0

In [6]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temps = [300,315,330,345,360,375,390]
rep = range(len(temps))
pressure    = None

step_size       = 0.01*unit.picoseconds
num_steps       = 20000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

friction   = 1.0/unit.picosecond

platform_name = 'CPU'  
platform = mm.Platform.getPlatformByName(platform_name)

contexto = []


for T in temps:
    integrator_T = mm.LangevinIntegrator(T, friction, step_size)
    times_T = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)
    positions_T  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)
    velocities_T = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)
    potential_energies_T = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
    kinetic_energies_T   = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)

    initial_positions_T  = np.zeros([n_particles, 3], np.float32) * unit.nanometers
    initial_velocities_T = None 

    context_T = mm.Context(system, integrator_T, platform)
    context_T.setPositions(initial_positions_T)
    context_T.setVelocitiesToTemperature(T)

    state_T = context_T.getState(getEnergy=True, getPositions=True, getVelocities=True)
    times_T[0] = state_T.getTime()
    positions_T[0] = state_T.getPositions()[0]
    velocities_T[0] = state_T.getVelocities()[0]
    kinetic_energies_T[0]=state_T.getKineticEnergy()
    potential_energies_T[0]=state_T.getPotentialEnergy()

    for ii in tqdm(range(num_steps_saved)):
        context_T.getIntegrator().step(saving_period)
        state_T = context_T.getState(getEnergy=True, getPositions=True, getVelocities=True)
        times_T[ii] = state_T.getTime()
        positions_T[ii] = state_T.getPositions()[0]
        velocities_T[ii] = state_T.getVelocities()[0]
        kinetic_energies_T[ii]=state_T.getKineticEnergy()
        potential_energies_T[ii]=state_T.getPotentialEnergy()
 

100%|██████████| 200/200 [00:03<00:00, 54.88it/s]


In [24]:
reps = []
reps.append(int(input("Escribe T inicial:")))
reps.append(int(input("Escribe la temperatura máxima:")))
reps.append(int(input("Escribe el incremento de T entre réplicas:")))

np.arange(reps[0],reps[1]+reps[2],reps[2])
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
pressure    = None
step_size       = 0.01*unit.picoseconds
num_steps       = 20000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

friction   = 1.0/unit.picosecond

platform_name = 'CPU'  
platform = mm.Platform.getPlatformByName(platform_name)

Escribe T inicial: 300
Escribe la temperatura máxima: 350
Escribe el incremento de T entre réplicas: 5


In [25]:
print(reps)

[300, 350, 5]


array([300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350])